In [11]:
from constants import *
from bit import *
from pathlib import Path
from web3 import Web3, HTTPProvider
from eth_account import Account
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
mnemonic=os.getenv("mnemonic")

In [34]:
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [51]:
coins = {
    ETH: derive_wallets(coin = ETH),
    BTCTEST: derive_wallets(coin = BTCTEST),
}
print(coins)

{'eth': [{'path': "m/44'/60'/0'/0/0", 'address': '0x02194a55DDA1B029F70220D53f099F38c629ab36', 'xprv': 'xprvA2UAbkzcn2ntbeUWWeryfF3uDhgNm4dauGWnvnDix6CsvCzQuvLXjSRM7JTtTTWK9sX95nFVJwYhVGxXqPToYqhz27ChLeVmsCQQXUKHxdM', 'xpub': 'xpub6FTX1GXWcQMBp8YycgPz2NzdmjWsAXMSGVSPjAdLWRjro1KZTTenHEjpxbBjGGK7wisPZe4htBommydMu8aPzi3Y94xHEdcN3ANiCP23RiY', 'privkey': '0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc', 'pubkey': '03b21e2e1bc0b74c298d113c5c305c80cc76e67c4ff29e9b31028ea17328f8bba8', 'pubkeyhash': 'a85e006b705f905c3e0e82485a33bf5bae8da52e', 'index': 0}, {'path': "m/44'/60'/0'/0/1", 'address': '0x3dFb64A5AdAe2aDece2E252FE5E889d61Ef75122', 'xprv': 'xprvA2UAbkzcn2ntezNrVAWRNF7yPRSuWDGEu7CdrwoNhYEjuJdeqhFC9zpZ9Uomp4Tuzf47cQ661EzPLErPP9nSvAHoboLqYTTpNd1xfeFGuUP', 'xpub': 'xpub6FTX1GXWcQMBsUTKbC3RjP4hwTHPufz6GL8EfLCzFsmin6xoPEZSho92znBTHFATi3RFHmhJLpwwR7P9KXcN9f4jAs4YUHRkkyg7spW5kZe', 'privkey': '0x81b180541f1ae73a48976095a58a342611fcedac9c00758510c0f3c0c63e5710', 'pubkey': '03b

In [46]:
def priv_key_to_account(coin, priv_key):
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    elif coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    
Eth_PrivateKey = coins["eth"][0]['privkey']
Btc_PrivateKey = coins['btc-test'][1]['privkey']
print(Eth_PrivateKey)
print(Btc_PrivateKey)

0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc
cVj1Q51vqgtBEYokpcb6RtjMwVbAsaCmrQRyjMTHFm5rGoziYe44


In [47]:
eth_account = priv_key_to_account(ETH,Eth_PrivateKey)
btc_account = priv_key_to_account(BTCTEST,Btc_PrivateKey)
print(eth_account)
print(btc_account)

<PrivateKeyTestnet: mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu>


In [48]:
def create_tx(coin, account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [49]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH: 
        trx_eth = create_tx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        trx_btctest= create_tx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest